Rough sketch of a proposal

- probably wrong for conjugate
- only optimized allocations and contract! (tensoradd! and tensortrace! were not modified)
- probably fails for non-symmetric tensormaps

In [1]:
using Revise, TensorKit, TensorOperations, BenchmarkTools, Strided, TupleTools, AllocationKit, MPSKitExperimental

In [3]:
import LinearAlgebra
LinearAlgebra.BLAS.set_num_threads(1)
Strided.set_num_threads(1)

1

In [4]:
using Base.Threads
nthreads()

8

# practical benchmarks

In [5]:
@tightloop_tensor ac_eff y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
@tightloop_tensor ac_eff_malloc allocator=malloc y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]

In [10]:
virtspace = Rep[SU₂](i => 20 for i in 0:10);
ospace = Rep[SU₂](0 => 5,1 => 2);
pspace = Rep[SU₂](1 => 1);

t_le = TensorMap(rand,ComplexF64,virtspace*ospace',virtspace);
t_re = TensorMap(rand,ComplexF64,virtspace*ospace,virtspace);
t_ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);
t_o = TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace);

factory = ac_eff(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));
factory_malloc = ac_eff_malloc(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));

In [11]:
@benchmark $factory_malloc(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 1867 samples with 1 evaluation.
 Range (min … max):  1.891 ms …  16.742 ms  ┊ GC (min … max): 0.00% … 70.86%
 Time  (median):     2.316 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.590 ms ± 909.134 μs  ┊ GC (mean ± σ):  1.06% ±  3.78%

   ▇█▅▆▆▇▇▅▄▂                                                  
  ▇███████████▇▅▆▃▄▃▂▂▃▂▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁ ▃
  1.89 ms         Histogram: frequency by time           5 ms <

 Memory estimate: 195.11 KiB, allocs estimate: 155.

In [12]:
@benchmark $factory(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 1307 samples with 1 evaluation.
 Range (min … max):  2.441 ms … 9.697 ms  ┊ GC (min … max):  0.00% … 31.48%
 Time  (median):     3.023 ms             ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.793 ms ± 1.590 ms  ┊ GC (mean ± σ):  14.60% ± 17.52%

   ▅█▃▄▁                                                     
  ▆██████▅▅▄▃▃▂▂▂▂▂▁▁▂▂▂▁▁▁▂▂▂▁▂▂▂▂▂▂▁▁▁▁▂▂▃▂▃▃▂▂▂▂▁▁▁▁▁▁▁▁ ▂
  2.44 ms        Histogram: frequency by time        8.3 ms <

 Memory estimate: 8.28 MiB, allocs estimate: 148.

In [13]:
function slowcontract(;le=nothing,re=nothing,x=nothing,O=nothing)
    @tensor y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
end
@benchmark slowcontract(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 979 samples with 1 evaluation.
 Range (min … max):  3.607 ms … 11.078 ms  ┊ GC (min … max):  0.00% … 41.56%
 Time  (median):     4.435 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.085 ms ±  1.487 ms  ┊ GC (mean ± σ):  12.67% ± 16.39%

    ▂▅▅▇▇█▇▅▅▁                                                
  ▃▄███████████▇▅▆▄▄▃▃▃▃▂▂▁▃▁▁▃▁▁▁▁▁▁▂▁▂▂▂▃▃▆▅▄▅▆▅▃▅▄▄▃▃▃▃▃▃ ▄
  3.61 ms        Histogram: frequency by time        8.78 ms <

 Memory estimate: 9.64 MiB, allocs estimate: 17248.

In [14]:
norm(slowcontract(le = t_le, re=t_re, x = t_ac, O = t_o)- factory(le = t_le, re=t_re, x = t_ac, O = t_o))

0.0

In [15]:
virtspace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 10 for i in 0:5,j in -3:3,b in 0:1);
ospace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 2 for i in 0:1,j in -1:1,b in 0:1);
pspace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 1 for i in 1:1,j in 0:0,b in 0:0);

t_le = TensorMap(rand,ComplexF64,virtspace*ospace',virtspace);
t_re = TensorMap(rand,ComplexF64,virtspace*ospace,virtspace);
t_ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);
t_o = TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace);

factory = ac_eff(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)))

ac_eff{TensorOperations.Backend{:none}, MPSKitExperimental.fast_init{GradedSpace{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, TensorKit.SortedVectorDict{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, Int64}}, 2, 2, ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, TensorKit.SortedVectorDict{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, Matrix{ComplexF64}}, FusionTree{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, 2, 0, 1, Nothing}, FusionTree{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, 2, 0, 1, Nothing}}, Tuple{MPSKitExperimental.fast_init{GradedSpace{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, TensorKit.SortedVectorDict{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, Int64}}, 2, 1, ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, TensorKit.SortedVectorDict{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, Matrix{ComplexF64}}, FusionTree{ProductSector{Tuple{SU2Irrep, U1Irrep, Z2Irrep}}, 2, 0, 1, Nothing}, FusionTree{ProductSector{Tuple{SU2Irrep, U1Irrep, Z

In [16]:
@benchmark $factory(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 147 samples with 1 evaluation.
 Range (min … max):  26.228 ms … 48.392 ms  ┊ GC (min … max): 0.00% … 8.76%
 Time  (median):     34.442 ms              ┊ GC (median):    9.45%
 Time  (mean ± σ):   33.389 ms ±  4.511 ms  ┊ GC (mean ± σ):  8.95% ± 8.13%

    ▅▅▂ ▆  ▂           ▂ ▅▃██  ▃  ▂▆                           
  ▇▅███▅████▅▅▅▅▁▄▅▄▇▅▅██████▇██▇▄██▁▅▄▅▄▁▁▄▁▁▁▁▁▁▁▄▄▁▁▁▁▁▁▁▄ ▄
  26.2 ms         Histogram: frequency by time        47.5 ms <

 Memory estimate: 53.59 MiB, allocs estimate: 691.

In [17]:
@benchmark slowcontract(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.280 s …   4.296 s  ┊ GC (min … max): 0.73% … 0.33%
 Time  (median):     4.288 s              ┊ GC (median):    0.53%
 Time  (mean ± σ):   4.288 s ± 10.719 ms  ┊ GC (mean ± σ):  0.53% ± 0.28%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.28 s         Histogram: frequency by time         4.3 s <

 Memory estimate: 238.81 MiB, allocs estimate: 1937790.

In [18]:
norm(slowcontract(le = t_le, re=t_re, x = t_ac, O = t_o)- factory(le = t_le, re=t_re, x = t_ac, O = t_o))

0.0

# multithreaded benchmark

gc becomes a severe bottleneck with many julia threads

In [19]:
virtspace = Rep[SU₂](0=>50, 1=> 30, 2=> 20, 3=>10, 4=> 5);
ospace = Rep[SU₂](0=>2, 1 => 2);
pspace = Rep[SU₂](1 => 1);


In [20]:
numblocks = 1;
const leftenvs = [TensorMap(rand,ComplexF64,virtspace*ospace',virtspace) for i in 1:numblocks];
const rightenvs = [TensorMap(rand,ComplexF64,virtspace*ospace,virtspace) for i in 1:numblocks];
const mpos = [TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace) for i in 1:numblocks];

ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);

In [21]:
using FLoops

t_le = leftenvs[1]
t_re = rightenvs[1]
t_o = mpos[1]

const fast_factory = ac_eff_malloc(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(ac),space(ac)))

function fastfun(x)
    @floop for (l,m,r) in zip(leftenvs,mpos,rightenvs)
        y = fast_factory(le = l,re = r, O = m, x = x)
        @reduce s += y
    end
    s
end

function slowfun(x)
    @floop for (le,O,re) in zip(leftenvs,mpos,rightenvs)
        @tensor y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
        @reduce s += y
    end
    s
end

slowfun (generic function with 1 method)

In [22]:
@benchmark fastfun(ac)

BenchmarkTools.Trial: 6172 samples with 1 evaluation.
 Range (min … max):  717.265 μs …   7.713 ms  ┊ GC (min … max): 0.00% … 82.37%
 Time  (median):     775.449 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   805.445 μs ± 216.337 μs  ┊ GC (mean ± σ):  0.62% ±  2.34%

    ▆█▆▅▂                                                        
  ▂▅█████▇▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▂▂ ▃
  717 μs           Histogram: frequency by time         1.42 ms <

 Memory estimate: 102.36 KiB, allocs estimate: 130.

In [23]:
@benchmark slowfun(ac)

BenchmarkTools.Trial: 2133 samples with 1 evaluation.
 Range (min … max):  1.574 ms … 10.572 ms  ┊ GC (min … max): 0.00% … 65.93%
 Time  (median):     2.134 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.332 ms ±  1.054 ms  ┊ GC (mean ± σ):  6.93% ± 11.36%

    ▆█▇▆▅▁                                                    
  ▃▇██████▇▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂ ▃
  1.57 ms        Histogram: frequency by time        8.15 ms <

 Memory estimate: 3.37 MiB, allocs estimate: 6266.